In [1]:
from datetime import datetime, date, timedelta
import pandas as pd
import geopandas as gpd
print('All imported')

All imported


In [2]:
papka = str(input())

 C:/Users/Sirius/Desktop/Тест единства


In [3]:
def is_time_between(begin_time, end_time, check_time):
    if begin_time < end_time:
        return check_time >= begin_time and check_time <= end_time
    else:
        return check_time >= begin_time or check_time <= end_time

In [4]:
def intersect_sessions(first, second, switch_mode_time=60, drop_speed=350, one_image_volume=1000):
    start1 = first[0]
    end1 = first[1]
    start2 = second[0]
    end2 = second[1]
    if int((start2 - end1).total_seconds()) < switch_mode_time:
        if (start1 == start2) and (end1 == end2):
            return 0
        elif (start1 < start2) and (end2 < end1):
            return 0
        else:
            start2 = end1 + timedelta(seconds=switch_mode_time)
            if int((end2 - start2).total_seconds()) >= int((one_image_volume * 8) / (drop_speed)):
                return [start2, end2]
            else:
                return 0
    else:
        return [start2, end2]

In [5]:
def intersect_capture_with_sessions(capture, session, future_session=True, capture_time=15, switch_mode_time=60, drop_speed=350, one_image_volume=1000):
    start_cap = capture[0]
    end_cap = capture[1]
    start_session = session[0]
    end_session = session[1]
    if future_session:
        if int((start_session - end_cap).total_seconds()) < switch_mode_time:
            if (start_cap == start_session) and (end_cap == end_session):
                return 0
            elif (start_cap < start_session) and (end_session < end_cap):
                return 0
            else:
                start_session = end_cap + timedelta(seconds=switch_mode_time)
                if int((end_session - start_session).total_seconds()) >= int((one_image_volume * 8) / (drop_speed)):
                    return [start_session, end_session]
                else:
                    return 0
        else:
            return [start_session, end_session]
    else:
        if int((start_cap - end_session).total_seconds()) < switch_mode_time:
            if (start_cap == start_session) and (end_cap == end_session):
                return 0
            elif (start_session < start_cap) and (end_cap < end_session):
                after = int((end_session - end_cap).total_seconds()) - switch_mode_time
                before = int((start_cap - start_session).total_seconds()) - switch_mode_time
                if after >= before:
                    if after >= int((one_image_volume * 8) / (drop_speed)):
                        return [end_cap + timedelta(seconds=switch_mode_time), end_session]
                    else:
                        return 0
                else:
                    if before >= int((one_image_volume * 8) / (drop_speed)):
                        return [start_session, start_cap - timedelta(seconds=switch_mode_time)]
                    else:
                        return 0                
            else:
                end_session = start_cap - timedelta(seconds=switch_mode_time)
                if int((end_session - start_session).total_seconds()) >= int((one_image_volume * 8) / (drop_speed)):
                    return [start_session, end_session]
                else:
                    return 0
        else:
            return [start_session, end_session]

In [16]:
def calculate_memory_restricts(memory_store, cell, drop_speed=350, capture_time=15, switch_mode_time=60, one_image_volume=1000, shooting_program_file=papka + '/corrected_shooting_points_1.xlsx', ppi_sessions_file=papka +'/ЗРВ/SESSION_PPI.xlsx', output_file=papka + '/result_data.xlsx'):
    connection_sessions = pd.read_excel(ppi_sessions_file)
    shooting_sessions = pd.read_excel(shooting_program_file)
    
    # Удаляем из DataFrame те съемки, которые пересекаются с огр. ЦУПа или находятся в интервале 66 секунд после предыдущей съемки
    for id, row in shooting_sessions.iterrows():
        if (not bool(row["IS_VALID"])) or (not bool(row["TM_DLT_VLD"])):
            shooting_sessions = shooting_sessions.drop(axis=0, index=id, inplace=False)
    
    drop_plan = []
    shooting_plan = []
    total_plan = []   
    for id, row in shooting_sessions.iterrows():
        capture_point_time = datetime.strptime(str(row["TIME"]), '%Y-%m-%d %H:%M:%S')
        shooting_plan.append([capture_point_time, capture_point_time + timedelta(seconds=capture_time)])
    
    # Составляем план сброса информации в порядке возрастания
    for id_ses, session in connection_sessions.iterrows():
        start_session = datetime.strptime(session["START"], '%Y-%m-%d %H:%M:%S')
        end_session = datetime.strptime(session["END"], '%Y-%m-%d %H:%M:%S')
        drop_plan.append([start_session, end_session])
    drop_plan.sort(key=lambda x: x[0])

    # План сессий без пересечений
    correct_drop_plan = [drop_plan[0]]
    for i in range(len(drop_plan)):  
        try:
            correct_drop_plan.append(intersect_sessions(drop_plan[i], drop_plan[i + 1], switch_mode_time=switch_mode_time, drop_speed=drop_speed, one_image_volume=one_image_volume))
        except:
            pass
    
    correct_drop_plan2 = [drop_plan[0]]
    for i in range(len(correct_drop_plan)):
        try:
            correct_drop_plan2.append(intersect_sessions(correct_drop_plan[i], correct_drop_plan[i + 1], switch_mode_time=switch_mode_time, drop_speed=drop_speed, one_image_volume=one_image_volume))
        except:
            pass
    
    # Общий план: съемка + сессии связи
    total_plan = correct_drop_plan2 + shooting_plan
    total_plan.sort(key=lambda x: x[0])
    
    # Проверка на пересечения съемок и сессий связи
    for _ in range(3):
        for i in range(len(total_plan)):
            if total_plan[i] != 0 and total_plan[i] in shooting_plan:
                try:
                    if total_plan[i - 1] not in shooting_plan:
                        total_plan[i - 1] = intersect_capture_with_sessions(total_plan[i], total_plan[i - 1], False)
                except:
                    pass
                try:
                    if total_plan[i + 1] not in shooting_plan:
                        total_plan[i + 1] = intersect_capture_with_sessions(total_plan[i], total_plan[i + 1], True)
                except:
                    pass
        for i in total_plan:
            if i == 0:
                total_plan.remove(0)
        total_plan.sort(key=lambda x: x[0])
    
    # print('CAPTURE: ' + str(len(shooting_plan) * 15))
    # res = 0
    # for i in range(len(total_plan)):
    #     if total_plan[i] not in shooting_plan:
    #         res += int((total_plan[i][1] - total_plan[i][0]).total_seconds())
    # print('DROP: ' + str(res))

    # for i in range(len(total_plan)):
        # print(str(total_plan[i]) + '\t --- ' + str('CAPTURE' if int((total_plan[i][1] - total_plan[i][0]).total_seconds()) == 15 else 'DROP'))
    
    time_start_process_column = []
    time_end_process_column = []
    process_column = []
    # Симуляция записи и сброса информации в ЗУ КА
    for i in range(len(total_plan)):
        # print(str(total_plan[i]) + '\t --- ' + str('CAPTURE' if total_plan[i] in shooting_plan else 'DROP'))
        if total_plan[i] in shooting_plan:
            if cell == (len(memory_store) - 16):
                # print('write cell 240')
                for o in range(15):
                    if memory_store[o] == 1:
                        # print(f'WARNING: IMAGE IN {o} CELL WAS DELETED, BUT NOT DROPPED')
                        process_column.append(f'INVALID DELETE CELL {o}')
                        time_start_process_column.append(None)
                        time_end_process_column.append(None)
                    elif memory_store[o] == 2:
                        # print(f'deleted cell {o}')
                        process_column.append(f'VALID DELETE CELL {o}')
                        time_start_process_column.append(None)
                        time_end_process_column.append(None)                        
                    memory_store[o] = 0
                if memory_store[cell + 14] == 1:
                    # print(f'WARNING: IMAGE IN {i + 14} CELL WAS DELETED, BUT NOT DROPPED')
                    process_column.append(f'INVALID DELETE CELL {cell + 14}')
                    time_start_process_column.append(None)
                    time_end_process_column.append(None)
                elif memory_store[cell + 14] == 2:
                    # print(f'deleted cell {cell + 14}')
                    process_column.append(f'VALID DELETE CELL {cell + 14}')
                    time_start_process_column.append(None)
                    time_end_process_column.append(None)
                memory_store[cell + 14] = 0
                
                memory_store[cell] = 1
                time_start_process_column.append(str(total_plan[i][0]))
                time_end_process_column.append(str(total_plan[i][1]))  
                process_column.append(f'WRITE CELL {cell}')
            elif cell == len(memory_store):
                # print('write cell 0')
                cell = 0
                memory_store[cell] = 1
                time_start_process_column.append(str(total_plan[i][0]))
                time_end_process_column.append(str(total_plan[i][1]))  
                process_column.append(f'WRITE CELL 0')
            else:
                # print(f'write cell {cell}')
                try:
                    if memory_store[cell + 14] == 1:
                        # print(f'WARNING: IMAGE IN {i + 14} CELL WAS DELETED, BUT NOT DROPPED')
                        process_column.append(f'INVALID DELETE CELL {cell + 14}')
                        time_start_process_column.append(None)
                        time_end_process_column.append(None)
                    elif memory_store[cell + 14] == 2:
                        # print(f'deleted cell {cell + 14}')
                        process_column.append(f'VALID DELETE CELL {cell + 14}')
                        time_start_process_column.append(None)
                        time_end_process_column.append(None)
                    memory_store[cell + 14] = 0
                except:
                    pass
                memory_store[cell] = 1
                time_start_process_column.append(str(total_plan[i][0]))
                time_end_process_column.append(str(total_plan[i][1]))  
                process_column.append(f'WRITE CELL {cell}')
            cell += 1
        else:
            for _ in range((int((total_plan[i][1] - total_plan[i][0]).total_seconds()) * drop_speed) // (one_image_volume * 8)):
                for o in range(len(memory_store)):
                    if memory_store[o] == 1:
                        memory_store[o] = 2
                        # print(f'drop cell {o}')
                        time_start_process_column.append(str(total_plan[i][0]))
                        time_end_process_column.append(str(total_plan[i][1])) 
                        process_column.append(f'DROP CELL {o}')
                        break
    data = dict(PROCESS=process_column, START=time_start_process_column, END=time_end_process_column)
    df = pd.DataFrame(data)
    df.to_excel(output_file, index=False)
    return [memory_store, cell]

In [17]:
# 0 - no data, 1 - written data, 2 - dropped data
store = [0] * 256
current_cell = 0

In [18]:
store, current_cell = calculate_memory_restricts(store, current_cell)
print(store)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
